# Single Family Parcels in the City of Los Angeles


For our group project, my team is researching the potential impact that SB8, SB9, and SB10 can make on the supply of new housing in the City of Los Angeles. One of our primary data sources will highlight existing single-famiy zoning conditions in the City of Los Angeles. This data source will highlight areas in which ADUs and smallplexes are permitted to be constructed. The zoning information for the City of Los Angeles can be found [here](https://geohub.lacity.org/datasets/zoning/explore?location=34.055956%2C-118.234564%2C12.90). 

Thus, in this notebook,we will be filtering the zoning information of the City of Los Angeles and running a spatial autocorrelation analysis to understand which neighborhoods in Los Angeles have the highest percentage/concentration of single family zones. We will use the LA Times neighborhood boundary as our location index. 

## 1. Importing Data

In [1]:
# to read and wrangle data
import pandas as pd

# to import data from LA Data portal
from sodapy import Socrata

# to create spatial data
import geopandas as gpd

# for basemaps
import contextily as ctx

# For spatial statistics
import esda
from esda.moran import Moran, Moran_Local

import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster,plot_moran_simulation

import libpysal as lps

# Graphics
import matplotlib.pyplot as plt
import plotly.express as px

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [4]:
#lets import our data
gdf = gpd.read_file('data/LA_Zoning.json')

DriverError: data/LA_Zoning.json: No such file or directory

## 2. Preliminary Exploration

I want to get an idea of what my data looks like. We will look at shape to see the number of rows and columns available to us, type to make sure mapshaper did not convert our geopanda file into something else, and head to get a better look at what data is available to us.

In [ ]:
#number of rows and columns 
gdf.shape

We have 58663 rows and 7 columms. 

In [ ]:
# double checking to make sure mapshaper did not change the data type
type(gdf)

Geopandas! We're good to go. 

In [ ]:
# what does my data table look like?
gdf.head()

Okay it looks like we have an object ID column followed by three different categorization categories, two shape/float columns, and a geometry/location column. 

In [ ]:
# here is a summary of the information we just looked at
gdf.info()

Now that we know what our data looks like, we can start analyzing our data

## 3. Data Values

I want to look at the value options/counts in our object columns.

### Zoning Summary 

In [ ]:
gdf['ZONE_SMRY'].value_counts()

Okay, so this column let's me know what land use type each zone has. I know that I want to look at the residential zones, so this could be useful.

### Zone Class

In the LA, the zone class is defined as: *the types of uses that are permitted on a property, including regulations related to building setbacks and minimum lot size requirements.* Let's take a look at the zone classes in this data set.

In [ ]:
gdf['ZONE_CLASS'].value_counts()

Now this column tells me the actual zoning classification of the parcel. Downside, there are 111 unique values in this column. Upside, I only need the single family zone classes, so I won't really need to look into all 111 unique classifications. However, I might want to look at all the different classifications within the residential land use parcels.

### Zoning Code

By looking at the data table generated above, I can tell that the `ZONE_CMPLT` is giving us the complete zoning code for zone. The complete code gives us extra information like how tall a building on a property can be or if the parcel fall in a planning overlay districts. Let's take a look at how many unique values we have. 

In [ ]:
gdf['ZONE_CMPLT'].value_counts()

There are 1941 unique values which is a lot. For the purposes of our research, we don't really need to know the entire zoning code of each parcel. All we need to know is the actual zone class. Thus, I can somewhat ignore this column. I won't delete it just yet because I may find use for it in the future.  

## 4. Filtering Data

### Residential Land Use 

I want to start off by filtering for the residential land use parcels only. This will help me trim the number of zone classes I can use.

In [ ]:
gdf.query("ZONE_SMRY == 'RESIDENTIAL'")

Now we have all the residential zones. This reduced our dataset from 58663 to 36534 rows. 

Let's define this query as *residential*

In [ ]:
residential = gdf.query("ZONE_SMRY == 'RESIDENTIAL'").copy()
residential

Now that I have filtered for residential land use, I can take a look at the residential zone classes to find the ones that apply to single family residential lots. 

In [ ]:
residential['ZONE_CLASS'].value_counts()

Now I can see exactly what the different zone class options for residntial land uses are. The single family zone classifications with the sufficient minimum rear yard setback requirement to build an ADU are: R1, RS, RU, R1V1, R1V2, RIV3, R1P, R1R3, R1H1, R1P, RW1, RE9, RE11, RE15, RE20, RE40. ([source](https://planning.lacity.org/odocument/eadcb225-a16b-4ce6-bc94-c915408c2b04/Zoning_Code_Summary.pdf)). I double checked our zone class values listed above to make sure the the codes match. They do! Thus, I want to filter my data for those parcels.


### Filtering for Single Family Zones

In [ ]:
# I want to define a list of my filtered categories
filter_list = ['R1', 'RS', 'RU', 'R1V1', 'R1V2', 'RIV3', 'R1P', 'R1R3', 'R1H1', 'RW1', 'RE9', 'RE11', 'RE15', 'RE20', 'RE40']

In [ ]:
#Now I filter my data for single family residential zones using my filtered zone classes.
gdf[gdf.ZONE_CLASS.isin(filter_list)]

We now have 19,843 rows which means we have 19,843 single family zones. However, each of these zones actually has multiple parcels on them. Unfortunately, this data does not give us the count of parcels within each zone. We will look into the parcel data for the final.

I'm going to define this list so that we can easily analyze and visualize only our single family zones.

In [ ]:
#Define this list
single_family = gdf[gdf.ZONE_CLASS.isin(filter_list)].copy()
single_family

In [ ]:
# Let's check distribution 
single_family['ZONE_CLASS'].value_counts()

Here we can see the number of zones we have with each zone class. It's interesting to see the type of single family zoning that is the most and the least prominant. We see that R1 and RS zones are our most prominant zones. R1 is known as traditional single family zoning, so it makes sense that R1 zones make up the majority of our data. RS is our suburban single family zoning. Historically, LA suburbs are known to have a higher rate of single family zoning than urban neighborhood, so again it makes sense that RS zones would be the second most prominant. Now our least prominant zone class is RU. RU is single family urban zones. We know that single family zoning is not very prominant. However, we did not expect that there would only be 2 RU zones in the entire city. 

## 6. Mapping our Single Family Zones

Now we will map our data so that we can actually visualize the distribution of single family zoning throughout the city. It will be interesting to see which areas have a lot of single family zoning and which areas do not. 

We plot our data on a web basemap; however, before adding web map tiles to these plots, I need to ensure the coordinate reference systems (CRS) of the tiles and the data match. Web map tiles are typically provided in Web Mercator (EPSG 3857), so let us first check what CRS we are in.

Let's start with a map of all our single family zones.

In [ ]:
single_family.crs

Now we know the CRS do not match, so we need to choose in which CRS we wish to visualize the data: either the CRS of the tiles, the one of the data, or even a different one.

The first option to match CRS is to leverage the `to_crs` method of GeoDataFrames to convert the CRS of our data, here to Web Mercator:

In [ ]:
single_family_wm = single_family.to_crs(epsg=3857)

We can then use `add_basemap` function of contextily to easily add a background map to our plot:

ax = single_family_wm.plot(figsize=(10,10), color = 'darkblue')
ax.set_title('Single Family Zones in the City of LA', fontsize=15)
ax.axis('off')
ctx.add_basemap(ax)

Well that definetly looks like an outline of LA. We can also see that single family zoning is concentrated in some areas an not so much in others. It will be interesting to overlay an outline of LA city boundaries so we can get a better idea of where single family zoning is more prominant than others. 

Now let's do our single family zone code map. This way we can also geet an idea of where each zone code is most prominant.

In [ ]:
ax = single_family_wm.plot(
            figsize=(10,10),   
            column = 'ZONE_CLASS',  
            cmap = 'plasma', alpha = 1, legend = True,           
            legend_kwds={
               'loc': 'lower left',
            }        
) 
ax.axis('off')
ax.set_title('Single Family Zones in the City of LA by Zone Code', fontsize=15)
ctx.add_basemap(ax)

Okay, so we can clearly see that different zone codes are prominant in different areas. For example, R1 zoning, our traditional single family zoning, is dispersed throughout the city. It also looks like our various RE zones are located in our mountains. This makes sense since RE classifies single family estates zoning and estates in Los Angeles are known to be concentrated in the hills. 

We also see that although R1 zoning is dispersed throughout the City of LA, RS zoning is mostly found in the north east of the city. This makes sense since RS zoning is known as single family suburban zoning and the north west of the city is the suburbs. However, R1 is still also prominent here. It would be interesting to understand why the city needed to distinguish between R1 and RS zoning. Additionally, it would be interesting to see if there is more or less affordable housing where RS zoning is more prominant than R1 zoning. 

Although this map is great at summarizing our distribution, it is a little hard to see the smaller zones and zone classes. Thus, we use a function and a loop to map each zone class separately. 

## 7. Converting original gdf to webmercator

We want to run a spatial autocorellation analysis to analyse the concentration of single family zones in Los Angeles. However, before we do that, we should also converty our original gdf to webmerecator. 

In [ ]:
gdf.crs
gdf_wm = gdf.to_crs(epsg=3857)

In [ ]:
ax = gdf_wm.plot(
            figsize=(10,10),   
            column = 'ZONE_CLASS',  
            cmap = 'magma', alpha = 1, legend = False,           
            legend_kwds={
               'loc': 'lower left',
            }        
) 
ax.axis('off')
ax.set_title('City of LA by Zone Code', fontsize=15)
ctx.add_basemap(ax)

## 8. Adding the LA Times Neighborhood Overlay

Now that our single family data and our original zoning data are compatibl with webmercator, we should add our LA Times Neighborhood overlay so that we have boundaries to run our spatial autocorellation on. We will use the same process we have used for our other data sources.

In [ ]:
Neighborhood_Boundaries = gpd.read_file('Data/LA_Times_Neighborhood_Boundaries.zip')

In [ ]:
Neighborhood_Boundaries.info()

In [ ]:
Neighborhood_Boundaries.sample(5)

In [ ]:
columns_to_keep_2 = ['name',
                     'geometry']

In [ ]:
Neighborhood_Boundaries = Neighborhood_Boundaries[columns_to_keep_2]

In [ ]:
Neighborhood_Boundaries.sample(5)

In [ ]:
Neighborhood_Boundaries.plot()

In [ ]:
Neighborhood_Boundaries = Neighborhood_Boundaries.to_crs(epsg=3857)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

Neighborhood_Boundaries.plot(ax=ax,
         color='blue', 
         edgecolor='black',
         lw=0.5,
         alpha=0.4)

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

## 9. Merging Data

We then overlayed zoning data on top of the neighborhood dataset

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

Neighborhood_Boundaries.plot(ax=ax,
        color='blue', 
        edgecolor='black',
        alpha=0.3)

single_family_wm.plot(ax=ax,
            color='green',
            markersize=10,
            alpha=1)


ax.axis('off')

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

Now lets combine our single family data with our neighborhood boundaries.

In [ ]:
Combined_Data = gpd.sjoin(single_family_wm, Neighborhood_Boundaries,how='left')

Combined_Data.head(3)

Now we combine our original total zoning data with our neighborhood boundaries. This will allow us to count the total number of zones in dataset so that we can calculate the percentage of single family zones to total zones in each neighborhood to run our spatial autocorrelation.

In [ ]:
All_Data = gpd.sjoin(gdf_wm, Neighborhood_Boundaries,how='left')

All_Data.head(3)

## 10. Calculating The Proportion of Single Family Zones in Each Neighborhood

We then identified which neighborhoods had the highest percentage/proportion of single family zones to total zones.

First we count the number of single family zones in each neighborhood. 

In [ ]:
Zones_By_Neighborhood = Combined_Data.name.value_counts().rename_axis('name').reset_index(name='single family zones')

In [ ]:
Zones_By_Neighborhood

Up top we see the 5 neighborhoods with the most and least single family zones, but we want to normalize this data. Thus, we will now count the number of total zones in each neighborhood to then calculate the percentages.

In [ ]:
Total_By_Neighborhood=All_Data.name.value_counts().rename_axis('name').reset_index(name='total zones')

In [ ]:
Total_By_Neighborhood

Lets add our counts (both single family and total) to our neighborhood boundaries data.

In [ ]:
Neighborhood_Boundaries=Neighborhood_Boundaries.merge(Zones_By_Neighborhood,on='name')

In [ ]:
Neighborhood_Boundaries.sample(10)

In [ ]:
Neighborhood_Boundaries=Neighborhood_Boundaries.merge(Total_By_Neighborhood,on='name')

In [ ]:
Neighborhood_Boundaries.head(10)

Now let's calculate the percentage of single family zones in each neighborhood by dividing the number of single family zones by total zones and multiplying by 100. 

In [ ]:
Neighborhood_Boundaries['percent_single_family'] = Neighborhood_Boundaries['single family zones']/Neighborhood_Boundaries['total zones']*100


In [ ]:
Neighborhood_Boundaries.sort_values(by="percent_single_family").tail()

In [ ]:
Neighborhood_Boundaries.round(2)

In [ ]:
Neighborhood_Boundaries.round(2)
Neighborhood_Boundaries = Neighborhood_Boundaries.round(2)

## 11. Mapping our Normalized Data

I then mapped the 20 neighborhoods with the most single family zones and the 20 neighborhoods with the least.

In [ ]:
fig,ax = plt.subplots(figsize=(12,10))
Neighborhood_Boundaries.sort_values(by='percent_single_family',ascending=False)[:20].plot(ax=ax,
                                                                 color='green',
                                                                 edgecolor='white',
                                                                 alpha=0.5,legend=True)


# title
ax.set_title('20 Neighborhoods With the Highest Percentage of Single Family Zones')

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
fig,ax = plt.subplots(figsize=(12,10))
Neighborhood_Boundaries.sort_values(by='percent_single_family',ascending=True)[:20].plot(ax=ax,
                                                                 color='red',
                                                                 edgecolor='white',
                                                                 alpha=0.5,legend=True)


# title
ax.set_title('20 Neighborhoods With the Lowest Percentage of Single Family Zones')

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 10), sharey=True, sharex=True,)
ax1, ax2, = axs

Neighborhood_Boundaries.sort_values(by='percent_single_family',ascending=False)[:20].plot(ax=ax1,
                                                                 color='green',
                                                                 edgecolor='white',
                                                                 alpha=0.5,legend=True)
ax1.axis('off')
ax1.set_title('20 Neighborhoods With the Highest Percentage of Single Family Zones', fontsize=10)
ctx.add_basemap(ax1)

Neighborhood_Boundaries.sort_values(by='percent_single_family',ascending=True)[:20].plot(ax=ax2,
                                                                 color='red',
                                                                 edgecolor='white',
                                                                 alpha=0.5,legend=True)
ax2.axis('off')
ax2.set_title('20 Neighborhoods With the Lowest Percentage of Single Family Zones', fontsize=10)
ctx.add_basemap(ax2)



We made a chloropleth map to show the relationship that neighborhoods with the largest and smallest percentages of single family zones.

In [ ]:
fig,ax = plt.subplots(figsize=(15,15))

Neighborhood_Boundaries.plot(ax=ax,
        column='percent_single_family',
        legend=True,
        alpha=0.8,
        cmap='plasma',
        scheme='natural_breaks')

ax.axis('off')
ax.set_title('Concentration of Single Family Zones in LA Neighborhoods',fontsize=22)
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

As expected we see the highest number of single family zones in suburban LA and the lowest number of single family zones in urban LA. For the final, we will overlay this data with affordability indicators. 

## 12. Global Spacial Autocorrelation

Now that we have combined all our data and found our normalized proportions, we can run our global spacial autocrrelation analysis. Let's add our spatial weights and caluculate our spatial lags.

In [ ]:
# calculate spatial weight
wq =  lps.weights.KNN.from_dataframe(Neighborhood_Boundaries,k=8)

# Row-standardization
wq.transform = 'r'

### Spatial Lag

In [ ]:
# create a new column for the spatial lag
Neighborhood_Boundaries['percent_single_family_lag'] = lps.weights.lag_spatial(wq, Neighborhood_Boundaries['percent_single_family'])

In [ ]:
Neighborhood_Boundaries.sample(10)

The spatial lag is a "a calculated assignment to each geography in your data, which takes into account the data values from others in their  "neighborhood" as defined by the spatial weight."

### The donut and the diamond

In [ ]:
# a column that calculates the difference betwen single family zones and lag
Neighborhood_Boundaries['single_family_diff'] = Neighborhood_Boundaries['percent_single_family'] - Neighborhood_Boundaries['percent_single_family_lag']

In [ ]:
Neighborhood_Boundaries.sort_values(by='single_family_diff')

In [ ]:
# the neighborhood with highest negative difference
gdf_donut = Neighborhood_Boundaries.sort_values(by='single_family_diff').head(1)
gdf_donut

Our Donut is Hollywood. This means that Hollywood has a low percentage of single family zones but is surrounded by neighborhoods with a high percentage of single family zones. This makes a lot of sense. Hollywood is generally a tourist neighborhood with a lot of commercial and industrial buildings. When you drive through Hollywood you mostly see multifamily homes. However, Hollywood is also adjacent to neighborhoods inn the Hollywood Hills that are almost completely single family zones and Hancock Park which is clustered with large single family mansions.

In [ ]:
# the neighborhood with highest positive difference
gdf_diamond = Neighborhood_Boundaries.sort_values(by='single_family_diff').tail(1)
gdf_diamond

Our Diamond is Mount Washington which means Mount Washington has a high concentration of single family zones but is surrounded by neighborhoods with a low concentration of single family zones. This also makes sense given the history and location of redlining in Los Angeles. As a quick summary of it all, during redlining in the 30s and 40s Mount Washington was considered the sole "desirable" neighborhood surrounded by many "undersirable" (yellow) and "hazardous" (red) neighborhoods such as Lincoln Heights and Montecito Heights. Historically, redlined neighoborhoods remained urban with very few single family zones and homes while desirable neighborhoods (like Mount Washington) were suburban with a high concentration of single family zones. Our data is literally showing us the legacy redlining still has on the city nearly 80 years later. 

We also went ahead and mapped our donut (in red) and diamond (in green) below.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

Neighborhood_Boundaries.plot(ax=ax,
        color='blue', 
        edgecolor='black',
        alpha=0.3)

gdf_diamond.plot(ax=ax,
         color='green', 
         edgecolor='black',
         lw=0.5,
         alpha=0.8)

gdf_donut.plot(ax=ax,
         color='red', 
         edgecolor='black',
         lw=0.5,
         alpha=0.8)

ax.axis('off')

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

### Morans Plot

Now we use a Morans Plot to quantify the degree of spatial correlations of single family homes and neighborhoods. To begin this process, we test for global autocorrelation for a continuous attribute (single family zones).

In [ ]:
y = Neighborhood_Boundaries.percent_single_family
moran = Moran(y, wq)
moran.I

Our Moron value is roughly .13. This really only tells us the slope of our Moran scatterplot (pictured below). However, a positive slope tells us there is a positive correlation. To truly understand whether or not the location of single family zones is radomized, we need the p-value. Which we find using the next 3 codes.

In [ ]:
fig, ax = moran_scatterplot(moran, aspect_equal=True)
plt.show()

In [ ]:
plot_moran_simulation(moran,aspect_equal=False)

Now let's calculate our p-value! 

In [ ]:
moran.p_sim

A p-value of 0.006 is extremely low. In simplified terms a p-value this low allows us to reject the notion (null hypothesis) that the location of single family zoning througout LA is random.

In more complicated terms:
The value is calculated as an empirical P-value that represents the proportion of realisations in the simulation under spatial randomness that are more extreme than the observed value. A small enough p-value associated with the Moran’s I of a map allows to reject the hypothesis that the map is random. In other words, we can conclude that the map displays more spatial pattern than we would expect if the values had been randomly allocated to a locations.

If we generated a large number of maps with the same values but randomly allocated over space, and calculated the Moran’s I statistic for each of those maps, only 0.6% of them would display a larger (absolute) value than the one we obtain from the observed data, and the other 99.4% of the random maps would receive a smaller (absolute) value of Moran’s I. 

### 13. Local Spatial Autocorrelation

So far, we have only determined that there is a positive spatial autocorrelation between the concentration of single family zones in LA city and their locations. But we have not detected where clusters are. Local Indicators of Spatial Association (LISA) is used to do that. LISA classifies areas into four groups: high values near to high values (HH), Low values with nearby low values (LL), Low values with high values in its neighborhood, and vice-versa.

- HH: high single family zone concentration near other high single family zone concentration
- LL: low single family zone concentrationnear other low single family zone concentration
- LH (donuts): low single family zone concentration surrounded by high single family zone concentration
- HL (diamonds): high single family zone concentration surrounded by single family zone concentration

In [ ]:
# calculate local moran values
lisa = esda.moran.Moran_Local(y, wq)

In [ ]:
# Plot
fig,ax = plt.subplots(figsize=(15,15))
moran_scatterplot(lisa, ax=ax, p=0.05)
ax.set_xlabel("Percent of Single Family Zones")
ax.set_ylabel('Spatial Lag of Percent of Single Family Zones')

# add some labels
plt.text(1.95, 0.5, "HH", fontsize=15)
plt.text(1.95, -1.05, "HL", fontsize=15)
plt.text(-2, 0.5, "LH", fontsize=15)
plt.text(-2, -1.05, "LL", fontsize=15)
plt.show()

In the scatterplot above, the colored dots represents the rows (neighborhoods) that have a P-value less that 0.05 in each quadrant. In other words, these are the statisticaly significantly, spatially autocorrelated geographies.

### Spatial Autocorrelation Map
Finally, you can visually these statistically significant clusters using the `lisa_cluster` function:

In [ ]:
fig, ax = plt.subplots(figsize=(14,12))
lisa_cluster(lisa, Neighborhood_Boundaries, p=0.05, ax=ax)
plt.show()

Our HH (high near high) is in the mountains (known for single family zoning) and our LL is clustured near urban Downtown LA and where historically redlined. 

And create a map comparing different p-values

In [ ]:
# create the 1x2 subplots
fig, ax = plt.subplots(1, 2, figsize=(20, 20))

# regular count map on the left
lisa_cluster(lisa, Neighborhood_Boundaries, p=0.05, ax=ax[0])

ax[0].axis("off")
ax[0].set_title("P-value: 0.05")

# spatial lag map on the right
lisa_cluster(lisa, Neighborhood_Boundaries, p=0.01, ax=ax[1])
ax[1].axis("off")
ax[1].set_title("P-value: 0.01")

plt.show()

## Plotly

In [ ]:
token = 'pk.eyJ1IjoieW9obWFuIiwiYSI6IkxuRThfNFkifQ.u2xRJMiChx914U7mOZMiZw'
px.set_mapbox_access_token(token)

In [ ]:
Neighborhood_Boundaries = Neighborhood_Boundaries.to_crs('epsg:4326')

In [ ]:
fig = px.choropleth_mapbox(Neighborhood_Boundaries,
                   geojson=Neighborhood_Boundaries.geometry,
                   locations=Neighborhood_Boundaries.index,
                   color="single family zones",
                     mapbox_style="satellite-streets",
                    hover_data=['name','single family zones','percent_single_family'],
                    center = {"lat": 34.02, "lon": -118.37},
                          height=500,
                          title= 'Concentration of Single Family Zones in LA Neighborhoods')
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [ ]:
fig.write_html("percentage_single_plotly.html")